In [ ]:
import pandas as pd 
import requests
import numpy as np
import re
import warnings
from pandas import json_normalize


<span style="color:#EB4545">
MARKDOWN is in red.
</span>

<span style="color:#EB4545">
The cell below checks to see if the first data file is already present
</span>

In [ ]:
import os.path

file_exists = os.path.exists("inst_full.csv")

print(file_exists)

<span style="color:#EB4545">
REQUIREMENT 1 - READ DATA IN FROM AN API (reading in a second data set is in Cell 11 below)
<br><br>
The cell below checks to see if you already have the data and if not sends GET requests to the API

Note - This does not check to ensure the file is current or free of errors.  If you would liek to update your file or you are concerned that your file has errors, simply delete your "inst_full.csv" file and a new, updated file will be created.

</span>

<span style="color:#EB4545">
REQUIREMENT 2 - CLEAN DATA (additional cleaning of data in cells 4, 12-26 and mergeing of data in cells 19-21)
<br><br>
The last 20 lines in the cell below (most of the code in the ELSE statement) cleans the data by checking for columns with mixed data types and converting the entire column to strings.  It converts the data type, but also displays an error to identify the affected columns.  This is a solution I found on stackoverflow that I wanted to implement in my code.

</span>

In [ ]:

import pandas


if file_exists == False:
    print("FILE DOES NOT EXIST")

    response_API = requests.get('https://banks.data.fdic.gov/api/institutions?limit=10000&offset=0')
    #print(response_API.status_code)

    response_API2 = requests.get('https://banks.data.fdic.gov/api/institutions?limit=10000&offset=10000')
    #print(response_API2.status_code)

    response_API3 = requests.get('https://banks.data.fdic.gov/api/institutions?limit=10000&offset=20000')
    #print(response_API3.status_code)

    print("Get request sucessful from API")


    data = response_API.json()
    #print(type(data))

    data2 = response_API2.json()
    #print(type(data2))

    data3 = response_API3.json()
    #print(type(data3))



    institution_df = json_normalize(data['data'])
    #print(institution_df.head())
    #institution_df.to_csv('inst.csv')

    institution_df2 = json_normalize(data2['data'])
    #print(institution_df2.head())
    #institution_df2.to_csv('inst2.csv')

    institution_df3 = json_normalize(data3['data'])
    #print(institution_df3.head())
    #institution_df3.to_csv('inst3.csv')
    
    print("Data from API converted to json")


    institution_df = institution_df.reindex(sorted(institution_df.columns), axis=1)
    institution_df2 = institution_df2.reindex(sorted(institution_df2.columns), axis=1)
    institution_df3 = institution_df3.reindex(sorted(institution_df3.columns), axis=1)



    full_data = [institution_df, institution_df2, institution_df3]
    df = pd.concat(full_data)
    df.columns = df.columns.str.lstrip("data.")
    df.to_csv('inst_full.csv')
    
    print("New file created, shape should be (27816, 168)")
    print(df.shape)
    

else:
    print("FILE ALREADY EXISTS")

    myfile = 'inst_full.csv'
    target_type = str  # The desired output type

    with warnings.catch_warnings(record=True) as ws:
        warnings.simplefilter("always")

        mydata = pandas.read_csv(myfile, sep="|", header=None)
        print("Warnings raised:", ws)
        # We have an error on specific columns, try and load them as string
        for w in ws:
            s = str(w.message)
            print("Warning message:", s)
            match = re.search(r"Columns \(([0-9,]+)\) have mixed types\.", s)
            if match:
                columns = match.group(1).split(',') # Get columns as a list
                columns = [int(c) for c in columns]
                print("Applying %s dtype to columns:" % target_type, columns)
                mydata.iloc[:,columns] = mydata.iloc[:,columns].astype(target_type)

    df = pd.read_csv('inst_full.csv')
    

<span style="color:#EB4545">
NOTE - .append is being deprecated - which is why I chose to use .concat instead

</span>

<span style="color:#EB4545">
REQUIREMENT 2 - CLEAN DATA
<br><br>
Reformating the column names so they are all lower case and spaces are replaced with an underscore

</span>

In [ ]:
#print(df.columns)
df.columns= df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '_')
#print(df.columns)

<span style="color:#EB4545">
Printing some information about the Data Set
</span>

In [ ]:
bank_count = len(df)
print("There are", bank_count, "institutuions with data on the FDIC's API.\n")

active = df['active'].value_counts()[0]
print("There are", active, "active institutions registered with the FDIC.\n")

inactive = df['active'].value_counts()[1]
print("There are", inactive, "inactive institutions registered with the FDIC.\n")

<span style="color:#EB4545">
Cells 6-10 create two new Data Frames to analyze information about active and inactvie institutions.  Then printing some interesting statistics about each.
</span>

In [ ]:
active_df = df[df.active == 0]
print(len(active_df))
#print(active_df.head())

In [ ]:
inactive_df = df[df.active == 1]
print(len(inactive_df))
#print(inactive_df.head())

In [ ]:
min = "{:,.2f}".format(active_df["asset"].min())
max = "{:,.2f}".format(active_df["asset"].max())
mean = "{:,.2f}".format(active_df["asset"].mean())

print("The minimum assets held by an active FDIC registered institution are ${} \n".format(min))
print("The maximum assets held by an active FDIC registered institution are ${} \n".format(max))
print("The average assets held by an active FDIC registered institution are ${} \n".format(mean))

In [ ]:
min = "{:,.2f}".format(inactive_df["asset"].min())
max = "{:,.2f}".format(inactive_df["asset"].max())
mean = "{:,.2f}".format(inactive_df["asset"].mean())

print("The minimum assets held by an inactive FDIC registered institution are ${} \n".format(min))
print("The maximum assets held by an inactive FDIC registered institution are ${} \n".format(max))
print("The average assets held by an inactive FDIC registered institution are ${} \n".format(mean))

In [ ]:
min = "{:,.2f}".format(df["asset"].min())
max = "{:,.2f}".format(df["asset"].max())
mean = "{:,.2f}".format(df["asset"].mean().round(2))

print("The minimum assets held by an FDIC registered institution are ${} \n".format(min))
print("The maximum assets held by an FDIC registered institution are ${} \n".format(max))
print("The average assets held by an FDIC registered institution are ${} \n".format(mean))

<span style="color:#EB4545">
REQUIREMENT 1 - READ DATA IN FROM A CSV FILE

</span>

In [ ]:
#To add banks with FDIC violations, the CSV file is from the FDIC website

absolute_path = os.path.abspath('')
relative_path = "ots-enforcement-order-listing.csv"
full_path = os.path.join(absolute_path, relative_path)
enforcment_df = pd.read_csv(full_path)
print(len(enforcment_df))
#print(enforcment_df.head())

<span style="color:#EB4545">
REQUIREMENT 2 - CLEAN DATA
<br><br>
This fixes any column names in the index by making them all lower case and replacing spaces with

</span>

In [ ]:
#print(enforcment_df.columns)
enforcment_df.columns= enforcment_df.columns.str.lower()
enforcment_df.columns = enforcment_df.columns.str.replace(' ', '_')
#print(enforcment_df.columns)

<span style="color:#EB4545">
REQUIREMENT 2 - CLEAN DATA
<br><br>
The next two cells remove improperly formatted docket numbers and then remove rows without docket numbers from the data frame

</span>

In [ ]:
enforcment_df['docket_number'] = np.where(enforcment_df['docket_number'].str.startswith('H'), '', enforcment_df['docket_number'])

In [ ]:
#create a new df so the old df is unaltered

#sorts the new df by docket number
enf_df1 = enforcment_df.sort_values('docket_number')

#converts emtpy strings to NaN and then drops rows that do not have a Docket Number
enf_df1['docket_number'].replace('', np.nan, inplace=True)
enf_df1.dropna(subset=['docket_number'], inplace=True)

#Gives the size and first columns of the new_enf_df
print(len(enf_df1.sort_values('docket_number')))
#print(enf_df1.head(5))

<span style="color:#EB4545">
REQUIREMENT 2 - CLEAN DATA
<br><br>
This drops duplicates so we can determine not only the total numnber of violations, but also the number of unique violations.

</span>

In [ ]:
#drops duplicates

short_enf_df = enf_df1.drop_duplicates(subset=['docket_number'], keep='last')

#Gives the size and first columns of the short_enf_df
print(len(short_enf_df))
#print(short_enf_df.head())

<span style="color:#EB4545">
REQUIREMENT 2 - CLEAN DATA
<br><br>
In the data one Bank may have more than onve violation date, and each violation date may have multiple violations.  The cells below allow us to consider total violations, remove instances of multiple violations resulting from a single event to see unique violations, and we can also see which banks have any violations (without consideration for how many violations the bank may have).

</span>

In [ ]:
violation_count_df = enf_df1.pivot_table(index = ['docket_number'], aggfunc ='size')
vc_df = violation_count_df.reset_index()
vc_df.columns = ['docket_number', 'total_violations']
#print(vc_df)

In [ ]:
unique_violation_count_df = enf_df1.pivot_table(index = ['docket_number', "issue_date"], aggfunc ='size')
uvc_df = unique_violation_count_df.reset_index()
uvc_df.columns = ['docket_number', 'issue_date', 'violations']
#print(uvc_df)

In [ ]:
final_uvc_df = uvc_df.pivot_table(index = ['docket_number'], aggfunc ='size')
fvc_df = violation_count_df.reset_index()
fvc_df.columns = ['docket_number', 'unique_violations']
#print(fvc_df)

<span style="color:#EB4545">
REQUIREMENT 2 - MERGE DATA
<br><br>
The next two cells perform data merges, to create the final violation data.

</span>

In [ ]:
violation_stats_df = vc_df.merge(fvc_df)
#print(violation_stats_df)

In [ ]:
final_enf_df = short_enf_df.merge(violation_stats_df)
#print(final_enf_df.head)

<span style="color:#EB4545">
REQUIREMENT 2 - MERGE DATA
<br><br>
This merges the final violation data with the main bank data.  It then creates a smaller data set with just the columns used for the visualizations.  The other columns were not removed earlier so that they can be used later for additional data mining.

</span>

In [ ]:
df['docket'] = df['docket'].astype(str)
final_df = df.merge(final_enf_df, how='left', left_on='docket', right_on='docket_number')
small_final_df = final_df[['asset', 'chrtagnt', 'clcode', 'effdate', 'fdicregn', 'name', 'trust', 'total_violations', 'unique_violations']].copy()
#print(small_final_df.head)

In [ ]:
small_final_df['total_violations'] = final_df['total_violations'].fillna(0)
small_final_df['unique_violations'] = final_df['unique_violations'].fillna(0)
small_final_df = small_final_df.reindex(columns = small_final_df.columns.tolist() + ['year', 'decade','violation', 'no_violation'])

<span style="color:#EB4545">
REQUIREMENT 2 - CLEAN DATA
<br><br>
The following cells clean data, ensuring proper type and turning years into decades.

</span>

In [ ]:
small_final_df['year'] = small_final_df['effdate'].str[-4:].astype(int)
#print(small_final_df.head())

In [ ]:
small_final_df['decade'] = (small_final_df['year'] / 10).astype(int) *10
#print(small_final_df.head())

<span style="color:#EB4545">
The next two cells turn the violation and no_violation columns into 0 or 1, so logical tests can be run.  All numbers above 1 are converted to 1.

</span>

In [ ]:
small_final_df['violation'] = small_final_df['total_violations']
small_final_df['violation'].where(small_final_df['violation'] == 0, 1, inplace=True)

In [ ]:
small_final_df['no_violation'] = small_final_df['violation'] - 1
small_final_df['no_violation'].where(small_final_df['no_violation'] == 0, 1, inplace=True)
print(small_final_df.head(500))

<span style="color:#EB4545">
The next cell creates two separate data sets for Banks with and without violations.  Added to potentially add visualizations specific to these groups but not currently used.

</span>

In [ ]:
no_violation_df = small_final_df[small_final_df.total_violations == 0]
violation_df = small_final_df[small_final_df.total_violations != 0]

print(no_violation_df.shape[0])
print(violation_df.shape[0])

<span style="color:#EB4545">
The next cell turns the data in json files and csv so it can be used in Tableau Public and/or exported for other use.

</span>

In [ ]:
small_final_df.to_json('final.json', orient='records', lines=True)
no_violation_df.to_json('no_violation.json', orient='records', lines=True)
violation_df.to_json('violation.json', orient='records', lines=True)

In [ ]:
small_final_df.to_csv('final.csv')
no_violation_df.to_csv('no_violation.csv')
violation_df.to_csv('violation.csv')

<span style="color:#EB4545">
REQUIREMENT 3 - VISUALIZE/PRESENT DATA -
<br><br>
Below is a Tableau visualization, specifically, a side by side bar chart comparing the TOTAL number of Banks with and without violations by year they were registered.  This visualization shows the number of Banks registed every year since 1970.  The right side does not show when the violations occur, it shows when the Bank was registered.  
<span>

<span style="color:#EB4545">
REQUIREMENT 5 - INTERPRET DATA -
<br><br>
Interpretation - Even though Bank Registrations have slowly declined since 1990, the number of banks with violations has continued to increase.  Newer Banks have more violations than older banks.
<span>

In [ ]:
%%html

<div class='tableauPlaceholder' id='viz1679791706378' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ba&#47;Banks_By_Date_Total&#47;YearbyTotal&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='Banks_By_Date_Total&#47;YearbyTotal' /><param name='tabs' value='yes' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ba&#47;Banks_By_Date_Total&#47;YearbyTotal&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1679791706378');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

<span style="color:#EB4545">
REQUIREMENT 3 - VISUALIZE/PRESENT DATA -
<br><br>
Below is a Tableau visualization, specifically, a side by side bar chart comparing the PERCENTAGE of Banks with and without violations by year they were registered.  While the first visualiztion shows the TOTAL NUMBER of Banks with and Without Violations in each 5 year period, this visualization shows the PERCENTAGE of Banks in each 5 year period. 
<span>

<span style="color:#EB4545">
REQUIREMENT 5 - INTERPRET DATA -
<br><br>
Interpretation - Almost 97% of violations have occured in Banks registered after 1990.  If you want to choose a bank with a disproportionally low number of violations, you should try to choose a bank that was in existence before 1990.
<span>

In [ ]:
%%html

<div class='tableauPlaceholder' id='viz1679788977317' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ba&#47;Banks_By_Date_Percent&#47;Yearby&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='Banks_By_Date_Percent&#47;Yearby' /><param name='tabs' value='yes' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ba&#47;Banks_By_Date_Percent&#47;Yearby&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1679788977317');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>


<span style="color:#EB4545">
REQUIREMENT 3 - VISUALIZE/PRESENT DATA -
<br><br>
This compares banks based on where they were registered, The Office of the Comprtoller of the Currency (OCC), the Office of Thrift Supervision (OTS), and Banks registered with their respective State. 
<span>

<span style="color:#EB4545">
REQUIREMENT 5 - INTERPRET DATA -
<br><br>
Interpretation - Banks originally registered with the Office of Thrift Supervision have a much higher violation rate than any other.  Only 10% of Banks were registered with the OTS, yet over %40 of violations occur in OTS registered banks.  The OTS was merged with the Office of the Comprtoller of the Currency in 2011.
<br><br>
Banks registered with their respective state have the lowes percent of violations. ALmost 65% of Banks are state registered, yet less than 25% of violations violations occur in State registered Banks.
<br><br>
If you have the choice, choose to bank at a State Bank and avoid any Bank that was registed with the OTS.  Banks registered with the Office of the Comptroller of the Currency have a slighty higher than expected percentage of violations, but not by an amount so high that you should avoid doing business with them.
<span>

In [ ]:
%%html

<div class='tableauPlaceholder' id='viz1679789695946' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ba&#47;Banks_By_Registration&#47;Charter&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='Banks_By_Registration&#47;Charter' /><param name='tabs' value='yes' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ba&#47;Banks_By_Registration&#47;Charter&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1679789695946');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='99%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

<span style="color:#EB4545">
REQUIREMENT 3 - VISUALIZE/PRESENT DATA -
<br><br>
This compares banks based on which FDIC region they are in. The FDIC has field offices that oversee all Banks in their region.
<span>

<span style="color:#EB4545">
REQUIREMENT 5 - INTERPRET DATA -
<br><br>
Interpretation - The Atlanta, Chicago, Dallas, and San Francisco regions have a number of violations proportional to the number of banks in each reagion.  New York has a dispropotionally high number of violations.  Kansas City has a disproportionally low number of violations.  If you are choosing a regional or national bank, you may want to avoid one registed in the New York region.
<span>

In [ ]:
%%html

<div class='tableauPlaceholder' id='viz1679789167814' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ba&#47;Banks_By_Region&#47;Region&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='Banks_By_Region&#47;Region' /><param name='tabs' value='yes' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ba&#47;Banks_By_Region&#47;Region&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1679789167814');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

<span style="color:#EB4545">
REQUIREMENT 3 - VISUALIZE/PRESENT DATA -
<br><br>
This compares banks based on whether they have a Trust Department.
<span>

<span style="color:#EB4545">
REQUIREMENT 5 - INTERPRET DATA -
<br><br>
Interpretation - Not all Banks have Trust Departments.  In fact, only 27% of banks have Trust Departments.  Banks without Trust Departments have a higher than expected number of violations and Banks WITH Trust Departments have a lower than expected number of violations.
<span>

In [ ]:
%%html

<div class='tableauPlaceholder' id='viz1679789192288' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ba&#47;Banks_By_Trust_Dept&#47;Trust&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='Banks_By_Trust_Dept&#47;Trust' /><param name='tabs' value='yes' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Ba&#47;Banks_By_Trust_Dept&#47;Trust&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1679789192288');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>


<span style="color:#EB4545">
REQUIREMENT 5 - INTERPRET DATA -
<br><br>
FINAL INTERPRETATION - 
<br><br>
There are several of the factors considered that have statistically significant correlations to Bank Violations.  These relationships are correlations, and not necessarily causation. However, when violations are not evely spread accross Banks, and appear at disproportionally higher rates in Banks in certain categories, it is cause for concern and further investigation.  Are any of the categories causal?  If not, what is the cause for the high correlation? These are questions outside the scope of this project, but are interesting topics for future consideration.
<br><br>
If you want to choose a Bank with the lowest chance of FDIC violations, you should choose an OLDER, STATE REGISTERED Bank, with a TRUST DEPARTMENT, that is registered in the Kansas City region.  
<br><br>
If you currently bank with an institution that was registered with the OTS, after 1990, has no trust dpeartment, and is in New York's region, you may want to consider switching.
<span>